<a href="https://colab.research.google.com/github/GVourvachakis/DeepDLT/blob/main/main.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" width="300" height="auto"/>
</a>

## Custom Components

### create_dataset.py 
Creates directory "./dataset" of **cropped and brightness-varying (by $\pm$ 20%) 64x64 images** (create_dataset() function) in BMP (bitmap) format for storage efficiency and their respective **csv files** (create_data_with_labels_csv() function) splitted as training, validation, and testing datasets sampled from the excel file "./images/all_images.xlsx".

### dataset_loader.py
Construct flexible/modular **custom dataset class** LaserDataset(Dataset) with Ordinal encoded "PP1" categorical feature and respective train/val/test dataloaders (prepare_and_load_data() function) .

### stratified_split.py
Contains a subroutine for k-fold label-wise cross-validation splitting (k_fold_cross_validation() function, with fold=5 as default) and the main execution/development of the folds under a multil-label cross-validation() splitting scheme (main_cross_validation() function) .

### training_pipeline.py 
Implements a complete training pipeline for an autoencoder that not only monitors basic loss values but also image quality metrics (PSNR and SSIM), while saving the best performing models according to each metric.

### label_training.py
Systematically evaluates how well the model performs on different types of input features using k-fold cross-validation, tracking performance metrics and providing visual feedback for each fold while maintaining separate results for each feature type.

### inference.py
Contains functions providing comprehensive visualization and analysis tools for examining how well autoencoder models are performing, both through numerical metrics and visual comparisons, while also enabling exploration of the VAE's generative capabilities (the latter one needs further development).

### train_vae.py
Trains a Variational Autoencoder (VAE) by iteratively processing data through training and validation phases, using either Adam or SGD optimizer. It saves checkpoints of the model's progress and keeps track of the best-performing version based on validation loss.

### environment.yml
contains all the dependencies and requirements.

### main.ipynb
Notebook where the whole training and inferencing pipeline is implemented 

### KFold_split.py
Creates DataLoader instances for 5-fold cross-validation . [not-used]

Connect into the custom virtual environment

In [1]:
import os
!source .venv/bin/activate
os.environ['VIRTUAL_ENV']

'/home/georgios-vourvachakis/Desktop/DeepDLT/.venv'

In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



Import native python and torch dependencies

In [3]:
import numpy as np ; import matplotlib; import pandas as pd
#import matplotlib.pyplot as plt
import subprocess
import tqdm
import torch ; import torchvision
#import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader
# from torchvision import datasets, transforms

Import quantitative reconstruction evaluation metrics via scikit-image

In [6]:
import skimage as ski
print(ski.__version__)

0.25.0


In [7]:
!python --version
print(f" matplotlib:\t{matplotlib.__version__}\n numpy:\t\t{np.__version__}\
      \n pandas:\t{pd.__version__}\n tqdm:\t\t{tqdm.__version__}\
      \n torch:\t\t{torch.__version__}\n torchvision:\t{torchvision.__version__}\
      \n skimage:\t{ski.__version__}")

Python 3.12.3
 matplotlib:	3.10.0
 numpy:		2.2.1      
 pandas:	2.2.3
 tqdm:		4.67.1      
 torch:		2.5.1+cu124
 torchvision:	0.20.1+cu124      
 skimage:	0.25.0


Import custom dependencies

In [9]:
from dataset_loader import prepare_and_load_data #, LaserDataset
from create_dataset import create_dir

# class CNNAutoencoder is exposed in __init__.py

# Directory tructure:
# DeepDLT/
# ├── models/
# │   ├── __init__.py
# │   └── autoencoder.py
#     └── vae.py
# └── autoencoder.ipynb.py
from models.autoencoder import CNNAutoencoder 

from training_pipeline import train_model, load_checkpoint
from inference import plotting, visualize_reconstruction

# for k-fold label stratification
from label_training import train_and_evaluate_kfold

# VAE components
from models.vae import CNNVariationalAutoencoder
from train_vae import train_vae

Construct directory of augmented images along with train/val/test csv datasets
(given the directory "./datasets" doesn't exist already)

In [10]:
# if not os.path.exists("./datasets"):
#     subprocess.run(["python", "create_dataset.py"])

Generate uniform label distribution-aware 5-fold cross-validation data (better *generalization*, acounting for *outliers*, and preventing *overfitting*) [given there are train/val/test files to sample from]

In [11]:
# if os.path.exists("./datasets/data_with_labels_csv"):
#     subprocess.run(["python", "stratified_split.py"])

**Complete preprocessing pipeline**:
create_dataset , data_with_labels_csv and globally create train/val/test Dataloaders

In [12]:
# Define paths
input_dirs = [
                '2020-4-30 tuning ripple period',
                '2020-6-9 Crossed polarized',
                'Paper Data/Double pulses',
                'Paper Data/Repetition 6p & 2p 29-4-2020',
                'Paper Data/Single pulses 2p',
                'Paper Data/Single pulses 4 and half 6',
                'Paper Data/Repetition 6p & 2p 29-4-2020/Details'
             ]
    
base_path = "./images"
excel_path = "./images/all_images.xlsx" # sample data for train/val/test csv files
csv_output_path = "./datasets/data_with_labels_csv"

dim = 64 # set dimensions of augmented images

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

workers = 2 if device=='cuda' else 4  # set the workers for the dataloaders. Suggested workers = 2 when on cuda

print(f"Using {device}, workers = {workers}")

images_path = f'./datasets/2023_im_dataset_{dim}x{dim}'
output_dir_images = create_dir(images_path)

train_loader, val_loader, test_loader = prepare_and_load_data(
                                                                input_dirs,
                                                                base_path,
                                                                output_dir_images,
                                                                excel_path,
                                                                csv_output_path,
                                                                cropped_dim=dim,
                                                                num_workers=workers,
                                                                batch_size=32
                                                             )   

Using cpu, workers = 4


Model initialization from "./DeepDLT/models" directory

In [ ]:
# Model Initialization
model = CNNAutoencoder(activation_function='relu', dropout_strength=0.3, filter_dim=5).to(device)

learning_rate = 1e-4
optimizer = 'Adam'
epochs = 2

# Train the model
train_losses, val_losses, psnr_values, ssim_values = train_model(model, train_loader, val_loader, device,
                                                                 optimizer=optimizer, num_epochs=epochs, learning_rate=learning_rate,
                                                                 checkpoint_name='model_checkpoint', # saving a checkpoint model every 10 epochs
                                                                 best_metric_checkpoint_name='best_model') # saving best models for loss, psnr, and ssim

## Loading Checkpoint for Inference and/or Resuming Training

In [ ]:
# Load checkpoint
file_path =  f'./models_history_{optimizer}/model_checkpoint.pt'

model, optimizer, start_epoch, loss = load_checkpoint(model, optimizer, file_path, lr=learning_rate)

# Set model to eval mode for evaluation or train mode to continue training
model.eval()  # For evaluation
# Or:
# model.train()  # For resuming training

print(f"Model restored to epoch {start_epoch} with loss {loss:.4f}")

In [ ]:
# Continue Training the model
train_losses, val_losses, psnr_values, ssim_values = train_model(model, train_loader, val_loader, device,
                                                                 optimizer='Adam', start_epoch=start_epoch, num_epochs=3, learning_rate=1e-2,
                                                                 checkpoint_name='model_checkpoint', 
                                                                 best_metric_checkpoint_name='best_model')

Obtain the loss curves, PSNR and SSIM values accross epochs

In [ ]:
plotting(train_losses=train_losses, val_losses=val_losses, psnr_values=psnr_values, ssim_values=ssim_values)

Visualize Reconstruction on Training and Testing Data

In [ ]:
# Reconstruction on Training Data
print("Reconstruction on Training Data:")
visualize_reconstruction(train_loader, model, model_label='Autoencoder', device=device, num_images=5)

# Reconstruction on Test Data
print("Reconstruction on Test Data:")
visualize_reconstruction(test_loader, model, model_label='Autoencoder', device=device, num_images=5)

## Label distribution-aware k-fold cross-validation section

In [ ]:
%%script false --no-raise-error
# Generate 5-fold splits across labels
if os.path.exists("./datasets/data_with_labels_csv"):
    subprocess.run(["python", "stratified_split.py"])

In [ ]:
%%script false --no-raise-error
#Output:

# Fold-wise train, validation, and test losses for each feature.
# Average test loss per feature.
# Printed comparison of reconstruction difficulty across features.

# Stratified splits path
stratified_dir = "./datasets/label_aware_splitting_data"
num_folds = 2 # for demonstration purposes

labels = ['angle', 'EP1', 'NP', 'PP1']

# Train and evaluate the model across all folds
# Parameters:
# model_class, fold_dir, num_folds, device, features, optimizer = "Adam",\
# num_workers=4, criterion=nn.MSELoss(), num_epochs=10, learning_rate=1e-3
train_and_evaluate_kfold(CNNAutoencoder, stratified_dir, num_folds, device, features=labels, num_workers=workers, num_epochs=1, learning_rate=1e-3)

## Variational Autoencoder Section

VAE Initialization

In [15]:
%%script false --no-raise-error

# VAE Parameters:
# activation_function: str = 'relu', dropout_strength: float = 0.2, latent_dim: int = 128
        
vae_model = CNNVariationalAutoencoder(activation_function='leakyrelu',dropout_strength=0.2).to(device)
learning_rate = 1e-4
optimizer = 'Adam'
epochs = 2 # for demonstration purposes

VAE's training initiation

In [16]:
%%script false --no-raise-error

# Parameters:
# model, train_loader, val_loader, device, optimizer: str = 'Adam', start_epoch=0, num_epochs=100,\
# learning_rate=1e-3, checkpoint_name='vae_checkpoint', best_metric_checkpoint_name='best_vae_model'
train_losses, val_losses = train_vae(vae_model, train_loader, val_loader, device,
                                     optimizer=optimizer, num_epochs=epochs, learning_rate=learning_rate,
                                     checkpoint_name='model_checkpoint', # saving a checkpoint model every 10 epochs
                                     best_metric_checkpoint_name='best_model') # saving best models for loss, psnr, and ssim

Image reconstruction evaluation

In [17]:
%%script false --no-raise-error

# Reconstruction on Training Data
print("Reconstruction on Training Data:")
visualize_reconstruction(train_loader, vae_model, model_label='Variational Autoencoder', device=device, num_images=5)

# Reconstruction on Test Data
print("Reconstruction on Test Data:")
visualize_reconstruction(test_loader, vae_model, model_label='Variational Autoencoder', device=device, num_images=5)
